<a href="https://colab.research.google.com/github/PRAGYAMISHRA04/Time-series-analysis/blob/main/Crypto_Research_TemporalCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.37.15.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.15.58:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


#**Importing Libraries**

In [2]:
"""! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download g-research-crypto-forecasting"""
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
! unzip "/content/drive/MyDrive/Crypto-research/train.csv (1).zip"

Archive:  /content/drive/MyDrive/Crypto-research/train.csv (1).zip
  inflating: train.csv               


In [4]:
from tensorflow.keras.layers import Concatenate, Add, GRU, GlobalAvgPool1D, Dense, Dropout, Input, Bidirectional, LSTM, Conv1D, Multiply 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsolutePercentageError , MeanSquaredError
from keras.utils.generic_utils import get_custom_objects
import math as m
import time
import gc
import os

In [5]:
"""def act_bipolar(x):
    '''Bipolar'''
    return ((1-tf.math.exp(-x)) / (1+tf.math.exp(-x)))
def act_SiLU(x):
    '''SiLU'''
    return (x/(1+tf.math.exp(-x)))
def act_softplus(x):
    '''Softplus'''
    return tf.nn.softplus(x)

"""
def act_LRELU(x):
  '''Leaky RELU'''
  return tf.nn.leaky_relu(x)
"""
def act_swish(x):
   '''Swish'''
   return x * tf.sigmoid(x)
"""
def act_Mcubic(x):
    '''Monotonic Cubic'''
    return (tf.math.pow(x,3) + x)

def act_NMcubic(x):
    '''Non Monotonic Cubic'''
    return (x - tf.math.pow(x,3))

def act_DSU(x):
    '''Decaying Sine Unit'''
    return (tf.constant(m.pi)/2)*(tf.experimental.numpy.sinc(x-tf.constant(m.pi)) - tf.experimental.numpy.sinc(x+tf.constant(m.pi))) 

def act_GCU(x):
    '''Growing Cosine Unit'''
    return x*tf.cos(x)

def act_shiftedSinc(x):
    '''Shifted Sinc'''
    return tf.constant(m.pi)*tf.experimental.numpy.sinc(x-tf.constant(m.pi))

def act_z2cosz(x):
  ''' x2cos(x)'''
  return x*x*tf.cos(x)

def act_sinz(x):
  ''' sin(x)'''
  return tf.sin(x)


"""
def act_Mish(x):
  '''Mish'''
  return x*tf.tanh(tf.math.log(1+tf.math.exp(x)))

def act_GELU(x):
  '''GELU'''
  return tf.keras.activations.gelu(x)
"""
def act_Shifted_Quadratic(x):
  '''Shifted Quadratic'''
  return (tf.math.pow(x,2) + x)
  
#get_custom_objects().update({'bipolar': Activation(act_bipolar)})
#get_custom_objects().update({'silu': Activation(act_SiLU)})
#get_custom_objects().update({'softplus': Activation(act_softplus)})
get_custom_objects().update({'lrelu': Activation(act_LRELU)})
#get_custom_objects().update({'swish': Activation(act_swish)})
get_custom_objects().update({'mcubic': Activation(act_Mcubic)})
get_custom_objects().update({'nmcubic': Activation(act_NMcubic)})
get_custom_objects().update({'dsu': Activation(act_DSU)})
get_custom_objects().update({'gcu': act_GCU})
get_custom_objects().update({'shiftedsinc': Activation(act_shiftedSinc)})
#get_custom_objects().update({'mish': Activation(act_Mish)})
#get_custom_objects().update({'gelu': Activation(act_GELU)})
get_custom_objects().update({'shiftedquadratic': Activation(act_Shifted_Quadratic)})
get_custom_objects().update({'z2cosz': Activation(act_z2cosz)})
get_custom_objects().update({'sin': Activation(act_sinz)})
"""class TimeCalcCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = time.time()    
    def on_epoch_end(self, epoch, logs=None):
        times.append(time.time() - self.epoch_time_start)"""

'class TimeCalcCallback(tf.keras.callbacks.Callback):\n    def on_epoch_begin(self, epoch, logs=None):\n        self.epoch_time_start = time.time()    \n    def on_epoch_end(self, epoch, logs=None):\n        times.append(time.time() - self.epoch_time_start)'

#**Importing Data**

In [6]:
dataset=pd.read_csv("train.csv")
dataset.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB


#**Data Preprocessing**

In [8]:
dataset.isnull().sum()

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
dtype: int64

In [9]:
dataset=dataset.dropna()

In [10]:
df=dataset.groupby("Asset_ID")
bitcoin_data=df.get_group(1)
bitcoin_data.shape

(1955978, 10)

In [11]:
bitcoin_data=bitcoin_data.drop(['Asset_ID','timestamp'],axis=1)

In [12]:
bitcoin_data.head()

,Count,Open,High,Low,Close,Volume,VWAP,Target
2,229.0,13835.194,14013.8,13666.11,13850.176,31.550062,13827.062093,-0.014643
10,235.0,13835.036,14052.3,13680.00,13828.102,31.046432,13840.362591,-0.015037
18,528.0,13823.900,14000.4,13601.00,13801.314,55.061820,13806.068014,-0.010309
26,435.0,13802.512,13999.0,13576.28,13768.040,38.780529,13783.598101,-0.008999
34,742.0,13766.000,13955.9,13554.44,13724.914,108.501637,13735.586842,-0.008079


In [13]:
def upper_shadow(df): return df['High'] - np.maximum(df['Close'], df['Open'])
def lower_shadow(df): return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df, row = False):
    df_feat = df
    df_feat['spread'] = df_feat['High'] - df_feat['Low']
    df_feat['mean_trade'] = df_feat['Volume']/df_feat['Count']
    df_feat['log_price_change'] = np.log(df_feat['Close']/df_feat['Open'])
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    df_feat['trade'] = df_feat['Close'] - df_feat['Open']
    df_feat['gtrade'] = df_feat['trade'] / df_feat['Count']
    df_feat['shadow1'] = df_feat['trade'] / df_feat['Volume']
    df_feat['shadow3'] = df_feat['upper_Shadow'] / df_feat['Volume']
    df_feat['shadow5'] = df_feat['lower_Shadow'] / df_feat['Volume']
    df_feat['diff1'] = df_feat['Volume'] - df_feat['Count']
    df_feat['mean1'] = (df_feat['shadow5'] + df_feat['shadow3']) / 2
    df_feat['mean2'] = (df_feat['shadow1'] + df_feat['Volume']) / 2
    df_feat['mean3'] = (df_feat['trade'] + df_feat['gtrade']) / 2
    df_feat['mean4'] = (df_feat['diff1'] + df_feat['upper_Shadow']) / 2
    df_feat['mean5'] = (df_feat['diff1'] + df_feat['lower_Shadow']) / 2
    df_feat['UPS'] = (df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open']))
    df_feat['UPS'] = df_feat['UPS']
    df_feat['LOS'] = (np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low'])
    df_feat['LOS'] = df_feat['LOS']
    df_feat['RNG'] = ((df_feat['High'] - df_feat['Low']) / df_feat['VWAP'])
    df_feat['RNG'] = df_feat['RNG']
    df_feat['MOV'] = ((df_feat['Close'] - df_feat['Open']) / df_feat['VWAP'])
    df_feat['MOV'] = df_feat['MOV']
    df_feat['CLS'] = ((df_feat['Close'] - df_feat['VWAP']) / df_feat['VWAP'])
    df_feat['CLS'] = df_feat['CLS']
    df_feat['LOGVOL'] = np.log(1. + df_feat['Volume'])
    df_feat['LOGVOL'] = df_feat['LOGVOL']
    df_feat['LOGCNT'] = np.log(1. + df_feat['Count'])
    df_feat['LOGCNT'] = df_feat['LOGCNT']
    df_feat["Close/Open"] = df_feat["Close"] / df_feat["Open"]
    df_feat["Close-Open"] = df_feat["Close"] - df_feat["Open"]
    df_feat["High-Low"] = df_feat["High"] - df_feat["Low"]
    df_feat["High/Low"] = df_feat["High"] / df_feat["Low"]
    if row: df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean()
    else: df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis = 1)
    df_feat["High/Mean"] = df_feat["High"] / df_feat["Mean"]
    df_feat["Low/Mean"] = df_feat["Low"] / df_feat["Mean"]
    df_feat["Volume/Count"] = df_feat["Volume"] / (df_feat["Count"] + 1)
    mean_price = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    median_price = df_feat[['Open', 'High', 'Low', 'Close']].median(axis=1)
    df_feat['high2mean'] = df_feat['High'] / mean_price
    df_feat['low2mean'] = df_feat['Low'] / mean_price
    df_feat['high2median'] = df_feat['High'] / median_price
    df_feat['low2median'] = df_feat['Low'] / median_price
    df_feat['volume2count'] = df_feat['Volume'] / (df_feat['Count'] + 1)
    return df_feat

In [14]:
data=get_features(bitcoin_data)
data.head()

,Count,Open,High,Low,Close,Volume,VWAP,Target,spread,mean_trade,...,High/Low,Mean,High/Mean,Low/Mean,Volume/Count,high2mean,low2mean,high2median,low2median,volume2count
2,229.0,13835.194,14013.8,13666.11,13850.176,31.550062,13827.062093,-0.014643,347.69,0.137773,...,1.025442,13841.3200,1.012461,0.987342,0.137174,1.012461,0.987342,1.012361,0.987244,0.137174
10,235.0,13835.036,14052.3,13680.00,13828.102,31.046432,13840.362591,-0.015037,372.30,0.132112,...,1.027215,13848.8595,1.014690,0.987807,0.131553,1.014690,0.987807,1.015958,0.989042,0.131553
18,528.0,13823.900,14000.4,13601.00,13801.314,55.061820,13806.068014,-0.010309,399.40,0.104284,...,1.029365,13806.6535,1.014033,0.985105,0.104087,1.014033,0.985105,1.013596,0.984680,0.104087
26,435.0,13802.512,13999.0,13576.28,13768.040,38.780529,13783.598101,-0.008999,422.72,0.089151,...,1.031137,13786.4580,1.015417,0.984755,0.088946,1.015417,0.984755,1.015504,0.984839,0.088946
34,742.0,13766.000,13955.9,13554.44,13724.914,108.501637,13735.586842,-0.008079,401.46,0.146229,...,1.029618,13750.3135,1.014951,0.985755,0.146032,1.014951,0.985755,1.015310,0.986103,0.146032


In [15]:
Scaler=MinMaxScaler()
data=Scaler.fit_transform(data)

In [16]:
size=data.shape[0]
train_size=m.floor(0.8*size)
val_size=m.floor(0.1*size)
train_data= data[: train_size] 
val_data = data[train_size: train_size+val_size ]
test_data = data[train_size+val_size:]

In [17]:
print('train:',train_data.shape)
print('valid:',val_data.shape)
print('test:',test_data.shape)

train: (1564782, 45)
valid: (195597, 45)
test: (195599, 45)


In [18]:
train_data

array([[0.0026064 , 0.17330815, 0.17557271, ..., 0.05821227, 0.95433888,
        0.03049393],
       [0.00267499, 0.17330559, 0.17619646, ..., 0.07515168, 0.96077376,
        0.02924381],
       [0.00602444, 0.17312497, 0.17535561, ..., 0.06402514, 0.94516068,
        0.02313585],
       ...,
       [0.02942488, 0.33103486, 0.33080052, ..., 0.00640812, 0.99769746,
        0.01035183],
       [0.02381197, 0.33109846, 0.33054696, ..., 0.00441304, 0.99740093,
        0.00722647],
       [0.02163997, 0.33085193, 0.33034769, ..., 0.00335411, 0.99739958,
        0.00834331]])

In [19]:
pd.DataFrame(train_data[0:2])

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0.002606,0.173308,0.175573,0.171341,0.173519,0.003032,0.173158,0.350256,0.034909,0.030483,...,0.050884,0.173407,0.067223,0.946803,0.030494,0.067223,0.946803,0.058212,0.954339,0.030494
1,0.002675,0.173306,0.176196,0.171566,0.173161,0.002984,0.173373,0.347877,0.037380,0.029230,...,0.054430,0.173529,0.079247,0.948759,0.029244,0.079247,0.948759,0.075152,0.960774,0.029244


In [20]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):  
        dataX.append(dataset[i:(i+time_step)] )
        dataY.append(dataset[i + time_step])
    return np.array(dataX), np.array(dataY)

# It shows RAM full for below line so i could not execute further with the calculated features. I once executes with single feature the whole thing ran so ig its only RAM issue. 

In [ ]:
train_X,train_Y = create_dataset(np.array(train_data),100)
val_X,val_Y=create_dataset(np.array(val_data),100)
test_X,test_Y=create_dataset(np.array(test_data),100)

In [ ]:
print('train_X:',train_X.shape)
print('validation_X:',val_X.shape)
print('test_X:',test_X.shape)


#**Model : Temporal CNN**

In [ ]:
! pip install keras-tcn
from tcn import TCN, tcn_full_summary

In [ ]:
NB_FILTERS = 16
DILATION = [2, 4, 8, 16]

In [ ]:
from tensorflow.keras.layers import Lambda , Masking , GlobalAveragePooling1D, Dense
from tensorflow.keras import Model, Sequential
import tensorflow_probability as tfp
# Concatenate , Add, GRU ,  Dropout, Input, Bidirectional, LSTM, Conv1D, Multiply
def Correlation(y_true,y_pred): return tf.math.abs(tfp.stats.correlation(y_pred,y_true, sample_axis=None, event_axis=None))

def masked_mae(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.mean_absolute_error(y_true = y_true_masked, y_pred = y_pred_masked)


In [ ]:
def Model_TCN(activations):
  tcn_layer = TCN(input_shape=(100, 1),return_sequences=True)
  #print('Receptive field size =', tcn_layer.receptive_field
  TCN_model= Sequential()
  TCN_model.add(Masking(mask_value=0.0,input_shape=(100,1)))
  TCN_model.add(tcn_layer)
  TCN_model.add(Activation(activations[0]))
  TCN_model.add(GlobalAveragePooling1D())
  TCN_model.add(Dense(128))
  TCN_model.add(Dense(64))
  TCN_model.add(Dense(1))
  TCN_model.compile(optimizer='adam', loss='mean_absolute_percentage_error', metrics= [Correlation, 'mean_absolute_percentage_error','mean_squared_error'])
  return TCN_model

In [ ]:
def train_and_test_model(activations,shapes,optimizers,i):
  test_metrics={}
  #checkpoint_path = "/content/model_{}/cp-{epoch:04d}.hd5".format(activations,epoch=0)
  #checkpoint_dir = os.path.dirname(checkpoint_path)
  #checkpoint=ModelCheckpoint(checkpoint_path.format(), monitor='val_mean_squared_error', verbose=0, save_best_only=False)
  #early=EarlyStopping(monitor='val_mean_squared_error', min_delta=0, patience=10, verbose=0, mode='auto')
  model=Model_TCN(activations) #,optimizer,activations
  history=model.fit(x=train_X,y=train_Y,
                    validation_data=(val_X,val_Y),
                    epochs=5,batch_size=1024) #checkpoint,early,         
  history = history.history
  model_eval=model.evaluate(test_X,test_Y,return_dict=True)
  p_test = model.predict(test_X)
  y_test_ = test_Y
  history.update({'activation_functions':len(history['val_mean_absolute_percentage_error'])*[activations],'epochs':list(range(1,len(history['val_mean_absolute_percentage_error'])+1))})
  data_list_test.append([activations,model_eval['mean_absolute_percentage_error'],model_eval['mean_squared_error']])
  del model
  return history

In [ ]:
shapes=[train_X.shape[1],train_X.shape[2]]
activation_strs=['LeakyReLU','PReLU','sigmoid','softplus','softsign','ReLU',
                  'gelu','selu','elu','tanh','gcu','swish',tfa.activations.mish,
                  'nmcubic','shiftedquadratic','z2cosz','sin','shiftedsinc','dsu','mcubic']
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3) #1e-4, 1e-5#
i=0

In [ ]:
data_list_test=[]
data_list=[]
i=0

In [ ]:
while i<20:
  print(" Activation Layer : ",activation_strs[i])
  activations=[activation_strs[i]]
  times = []
  history_ = train_and_test_model(activations,shapes,optimizer,i)
  #history_.update({'time_taken_on_epoch':times})
  data_list.append(pd.DataFrame(history_))
  gc.collect()
  i+=1

In [ ]:
data = pd.concat(data_list)
data_test = pd.DataFrame(data_list_test)

In [ ]:
data

In [ ]:
data_test

In [ ]:
data.to_csv('/content/Train_stats[GRESEARCH-TCN].csv',index=False)
data_test.to_csv('/content/Test_stats[GRESEARCH-TCN].csv',index=False)